https://arxiv.org/pdf/1503.04069.pdf

ソース

https://github.com/fomorians/lstm-odyssey

https://github.com/suriyadeepan/rnn-from-scratch


Investigating Gated Recurrent Neural Networks for Speech Synthesis

http://homepages.inf.ed.ac.uk/zwu2/demo/icassp16/lstm.html

LSTMのゲートの調整

Natural: Natural recording speech.<br>
LSTM: Baseline long short-term memory (LSTM) based RNN system.<br>
NIG: LSTM without input gate.<br>
NOG: LSTM without output gate.<br>
NFG: LSTM without forget gate.<br>
NPH: LSTM without peep-hole connections.<br>
GRU: Gated Recurrent Unit.<br>
S-LSTM: Simplified LSTM with only forget gate.<br>


In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import chainer
# from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import cuda, Function, Variable, optimizers, serializers, utils
from chainer import report, training, Chain, datasets, iterators, optimizers
# from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
import time
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# データ
def get_dataset(N):
    x = np.linspace(0, 2 * np.pi, N)
    y = np.sin(x)
    return x, y
print(get_dataset(10)[0].shape)
print(get_dataset(10)[1].shape)

(10,)
(10,)


In [ ]:
#VanillaLSTM
class RNN_Model(chainer.Chain):
    def __init__(self, n_units=10):
        super(RNN_Model, self).__init__(
            l1=L.Linear(1, n_units),
            l2=L.LSTM(n_units, n_units),
            l3=L.Linear(n_units, 1),
        )

    def reset_state(self):
        self.l2.reset_state()

    def __call__(self, x):
        x = Variable(x.reshape((len(x), 1)).astype(np.float32))
        h1 = self.l1(x)
        h2 = self.l2(h1)
        y = self.l3(h2)
        return y

    def  predict(self, x):
        x = x.reshape((len(x), 1)).astype(np.float32)
        x = Variable(x)        
        h1 = self.l1(x)
        h2 = self.l2(h1)
        y = self.l3(h2)
        return y

    def get_predata(self, x):
        return self.predict(x)
    
class LossFuncL(chainer.Chain):
    def __init__(self, predictor):
        super(LossFuncL, self).__init__(predictor=predictor)

    def __call__(self, x, t):
        y = self.predictor(x)
        x = x.astype(np.float32).reshape(len(x),1)
        t = t.astype(np.float32).reshape(len(t),1)
        loss = F.mean_squared_error(y, t)
        report({'loss':loss}, self)
        return loss
